## _*Particle hole transformation of FermionicOperator*_

This notebook demonstrates carrying out a ParticleHole transformation on the FermionicOperator in Qiskit Chemistry. Here we use the FermionicOperator directly to demonstrate.

Note: The Hamiltonian class that wraps this provides a means to use either full, or particle hole transformation. Under the covers it does what is shown here though.

This notebook has been written to use the PYSCF chemistry driver.

In [1]:
import numpy as np

from qiskit import BasicAer
from qiskit.transpiler import PassManager

from qiskit.aqua import Operator, QuantumInstance
from qiskit.aqua.algorithms.adaptive import VQE
from qiskit.aqua.algorithms.classical import ExactEigensolver
from qiskit.aqua.components.optimizers import L_BFGS_B
from qiskit.aqua.components.variational_forms import RY

from qiskit.chemistry import FermionicOperator
from qiskit.chemistry.drivers import PySCFDriver, UnitsType

We'll do this with H2 molecule and use the PySCF driver to create the integrals we need for the FermionicOperator.

In [3]:
driver = PySCFDriver(atom='H .0 .0 .0; H .0 .0 0.735', unit=UnitsType.ANGSTROM,
                     charge=0, spin=0, basis='sto3g')
molecule = driver.run()

We first create the FermionicOperator and use ExactEigensolver with qubit operator we get from it via a jordan wigner mapping to compute the ground state energy. Here this is the electronic component of the total ground state energy (the total ground state energy would include the nuclear repulsion energy we can get from the molecule that comes from the driver)

In [4]:
ferOp = FermionicOperator(h1=molecule.one_body_integrals, h2=molecule.two_body_integrals)
qubitOp_jw = ferOp.mapping(map_type='JORDAN_WIGNER', threshold=0.00000001)
qubitOp_jw.chop(10**-10)

# Using exact eigensolver to get the smallest eigenvalue
exact_eigensolver = ExactEigensolver(qubitOp_jw, k=1)
ret = exact_eigensolver.run()

# print(qubitOp_jw.print_operators())

print('The exact ground state energy is: {}'.format(ret['energy']))
print('The Hartree Fock Electron Energy is: {}'.format(molecule.hf_energy - molecule.nuclear_repulsion_energy))

The exact ground state energy is: -1.8572750302023795
The Hartree Fock Electron Energy is: -1.8369679912029842


Now the same as above but with ParticleHole transformation. This removes out energy from the FermionicOperator that is equivalent to the electronic part of the Hartree Fock Energy that we also computed above. The Hartree Fock energy also comes from the driver. To get the total electronic ground state energy we need to add the part we now compute with the part that was removed by the transformation.

In [5]:
# particle hole transformation
newferOp, energy_shift = ferOp.particle_hole_transformation(num_particles=2)
print('Energy shift is: {}'.format(energy_shift))
newqubitOp_jw = newferOp.mapping(map_type='JORDAN_WIGNER', threshold=0.00000001)
newqubitOp_jw.chop(10**-10)

exact_eigensolver = ExactEigensolver(newqubitOp_jw, k=1)
ret = exact_eigensolver.run()

# print(newqubitOp_jw.print_operators())
print('The exact ground state energy in PH basis is {}'.format(ret['energy']))
print('The exact ground state energy in PH basis is {} (with energy_shift)'.format(ret['energy'] - energy_shift))

Energy shift is: 1.8369679912029846
The exact ground state energy in PH basis is -0.020307038999396183
The exact ground state energy in PH basis is -1.8572750302023808 (with energy_shift)


We run here using the quantum VQE algorithm to show the same result. The parameters printed are the optimal parameters of the variational form at the minimum energy, the ground state.

In [6]:
# setup VQE 
# setup optimizer, use L_BFGS_B optimizer for example
lbfgs = L_BFGS_B(maxfun=1000, factr=10, iprint=10)

# setup variational form generator (generate trial circuits for VQE)
var_form = RY(newqubitOp_jw.num_qubits, 5, entangler_map = [[0, 1], [1, 2], [2, 3]])

# setup VQE with operator, variational form, and optimizer
vqe_algorithm = VQE(newqubitOp_jw, var_form, lbfgs, 'matrix')

backend = BasicAer.get_backend('statevector_simulator')
quantum_instance = QuantumInstance(backend, pass_manager=PassManager())

results = vqe_algorithm.run(quantum_instance)
print("Minimum value: {}".format(results['eigvals'][0].real))
print("Minimum value: {}".format(results['eigvals'][0].real - energy_shift))
print("Parameters: {}".format(results['opt_params']))

Minimum value: -0.020307038771711697
Minimum value: -1.8572750299746963
Parameters: [-0.62024568 -0.94461634 -0.12822854 -1.33174693 -3.12835752 -2.41119768
  0.67926104  2.44344768  0.72721421 -2.76518798 -1.08251803 -1.75962366
  0.54861203  1.8995056   3.04269648 -1.75046119  0.16409288  0.68204022
 -0.07661803 -0.76359574 -1.56412942 -2.02324628  1.50961019  1.31452025]
